Попробовала дообучить **rugpt3small_based_on_gpt2** на датасете **recipes 27к** c kaggle https://www.kaggle.com/datasets/coolonce/recipes-and-interpretation-dim

Модель была обучена командой SberDevices с длиной последовательности 1024 с использованием преобразователей на 80B токенах примерно в 3 эпохи.
Класс задач: Text Generation

Обучала на QA c использованием 100% данных колонок name(название рецепта) и Инструкции(описание рецепта), дообучила на 3 эпохи.
Этот вариант модлеи будет заготовкой одного из интентов (повар) для курсового проекта.

In [1]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 29.8 MB/s eta 0:00:00


In [2]:
!pip install transformers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.1 MB/s eta 0:00:00


In [3]:
!pip install datasets transformers==4.28.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 86.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.33.2
    Uninstalling transformers-4.33.2:
      Successfully uninstalled transformers-4.33.2


In [4]:
import numpy as np
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import torch
from sklearn.model_selection import train_test_split



In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [11]:
!ls "/content/drive/MyDrive/Data science_data/"

all_recepies_inter.csv


In [7]:
!ls "/content/drive/MyDrive/Data Science_models/nlp/project/"

model_qa_cats_gen  tokenizer_qa_cats_gen


In [12]:
path_d =  "/content/drive/MyDrive/Data science_data/"

In [9]:
path_m =  "/content/drive/MyDrive/Data Science_models/nlp/project/"

In [13]:
df_rec = pd.read_csv(path_d + 'all_recepies_inter.csv', sep='\t')

In [14]:
df_rec

,Unnamed: 0,name,composition,cooking_type,Инструкции,dish_type,Дата,photo,source,composition_inter
0,0,рассольник классический с перловкой и солеными...,"[{'Перловка': 0.1, 'unit': 'стак. (200 мл)'}, ...","варка,жарка",Подготовить указанные ингредиенты для приготов...,первое,05.06.2015,photo_1000menu_1.jpg,https://1000.menu/cooking/33395-rassolnik-s-pe...,"[{'product_id': 4253, 'name_source': 'Перловая..."
1,1,Суп пюре из белокочаной капусты,"[{'Капуста белокочанная': 50.0, 'unit': 'гр'},...",варка,"Необходимые ингредиенты\r\nНарезаем лук, морко...",первое,27.06.2015,photo_1000menu_2.jpg,https://1000.menu/cooking/25399-sup-pure-iz-be...,"[{'product_id': 2286, 'name_source': 'Капуста ..."
2,2,Постные щи из квашеной капусты,"[{'Капуста квашеная': 116.7, 'unit': 'гр'}, {'...","варка,жарка,тушение","Честно признаюсь, у меня не было репы на момен...",первое,12.02.2013,photo_1000menu_3.jpg,https://1000.menu/cooking/5159-postnje-shchi,"[{'product_id': 0, 'name_source': 'Капуста ква..."
3,3,Тюря- простой суп быстро и вкусно,"[{'Квас': 0.2, 'unit': 'л'}, {'Лук репчатый': ...",сырое,"\r\nНачинаем мы приготовление тюри с того, что...",первое,02.03.2011,photo_1000menu_4.jpg,https://1000.menu/cooking/5085-turya,"[{'product_id': 0, 'name_source': 'Квас', 'uni..."
4,4,Фасолевый суп из красной фасоли,"[{'Вода': 0.3, 'unit': 'л'}, {'Картошка': 0.3,...",варка,Подготовить ингредиенты. Для приготовления суп...,первое,28.01.2013,photo_1000menu_5.jpg,https://1000.menu/cooking/38765-fasolevyi-sup-...,"[{'product_id': 828, 'name_source': 'Вода', 'u..."
...,...,...,...,...,...,...,...,...,...,...
27879,26468,Салат с филе тунца и огурцами в азиатской…,"[{'Тунец': 50.0, 'unit': 'г'}, {'Огурцы': 50.0...",жарка,1. Для приготовления азиатского соуса смешать ...,салат,20.05.2015,photo_eda_28463.jpg,https://eda.ru/recepty/salaty/salat-s-file-tun...,"[{'product_id': 5989, 'name_source': 'Тунец', ..."
27880,26469,Теплый салат с куриной печенью и яблоками,"[{'Куриная печень': 120.0, 'unit': 'г'}, {'Кис...","варка,жарка,запекание","1. Маленькими ломтиками порезать яблоки, запек...",салат,27.09.2018,photo_eda_28464.jpg,https://eda.ru/recepty/salaty/teplyy-salat-s-k...,"[{'product_id': 0, 'name_source': 'Куриная печ..."
27881,26470,Самый зеленый салат,"[{'Мангольд': 'по вкусу', 'unit': 'по вкусу'},...",сырое,"1. Чеснок натереть, сыр фета раскрошить.\r\n2....",салат,24.03.2018,photo_eda_28466.jpg,https://eda.ru/recepty/salaty/samyy-zelenyy-sa...,"[{'product_id': 3378, 'name_source': 'Мангольд..."
27882,26471,Теплый салат с тыквой и брынзой под…,"[{'Тыква': 100.0, 'unit': 'г'}, {'Сыр брынза':...",запекание,"1. Тыкву нарезаем кубиками, добавляем оливково...",салат,14.05.2019,photo_eda_28467.jpg,https://eda.ru/recepty/salaty/teplyy-salat-s-t...,"[{'product_id': 6023, 'name_source': 'Тыква', ..."


In [15]:
data = df_rec[['Инструкции', 'name']]
data.dropna(subset=['Инструкции', 'name'], inplace=True)
data

<ipython-input-15-d1d14ddb4246>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(subset=['Инструкции', 'name'], inplace=True)


,Инструкции,name
0,Подготовить указанные ингредиенты для приготов...,рассольник классический с перловкой и солеными...
1,"Необходимые ингредиенты\r\nНарезаем лук, морко...",Суп пюре из белокочаной капусты
2,"Честно признаюсь, у меня не было репы на момен...",Постные щи из квашеной капусты
3,"\r\nНачинаем мы приготовление тюри с того, что...",Тюря- простой суп быстро и вкусно
4,Подготовить ингредиенты. Для приготовления суп...,Фасолевый суп из красной фасоли
...,...,...
27879,1. Для приготовления азиатского соуса смешать ...,Салат с филе тунца и огурцами в азиатской…
27880,"1. Маленькими ломтиками порезать яблоки, запек...",Теплый салат с куриной печенью и яблоками
27881,"1. Чеснок натереть, сыр фета раскрошить.\r\n2....",Самый зеленый салат
27882,"1. Тыкву нарезаем кубиками, добавляем оливково...",Теплый салат с тыквой и брынзой под…


In [16]:
train_q, test_q, train_ans, test_ans = train_test_split(data.name,
                                                        data.Инструкции,
                                                        test_size=.2,
                                                        random_state=42)

In [17]:
device = torch.device("cuda")

Модуль AutoModelForCausalLM и модуль AutoModelForQuestionAnswering являются двумя разными модулями, предназначенными для различных задач.

Модуль AutoModelForCausalLM предназначен для создания автоматически настраиваемой модели для генерации последовательностей, где последовательности имеют причинно-следственную связь. Например, такая модель может использоваться для генерации продолжения рассказа или прогнозирования следующего слова в текстовой последовательности.

С другой стороны, модуль AutoModelForQuestionAnswering предназначен для создания автоматически настраиваемой модели для ответа на вопросы на основе контекстных данных. Например, такая модель может использоваться для отвечения на вопросы в рамках задачи вопросно-ответной системы, в которой нужно найти ответ на вопрос в текстовом документе или корпусе данных.

In [18]:
model_name = 'sberbank-ai/rugpt3small_based_on_gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
!pip install pymorphy2
!pip install re
!pip install nltk
!pip install unicodedata
!pip install contractions
!pip install inflect
!pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 61.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=e9979ec08fa9a360c2dfb3641e96a88afad7a7ccd9e8e007e76df62aa7c81b19
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re
ERROR: Could not find a version that satisfies the requirement unicodedata (from versions: none)
ERROR: No matching distribution found for unicodedata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 7.2 MB/

In [20]:
!pip install num2words

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 2.2 MB/s eta 0:00:00


In [21]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#from nltk.stem import WordNetLemmatizer
import string
import pymorphy2
import re

import emoji
import unicodedata
import contractions
import inflect
from num2words import num2words




from nltk.corpus import stopwords
import pymorphy2

In [22]:
nltk.download('punkt')  # Токенизатор
nltk.download('stopwords')  # Стоп-слова
nltk.download('wordnet')  # Лемматизатор

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [23]:
def clean_text(input_text):
    """Очищает текст от HTML-тегов, URL-ссылок, эмоджи, приводит к нижнему регистру,
    удаляет специальные символы, записывает числа прописью, удаляет стоп-слова,
    лемматизирует слова и удаляет знаки препинания.

    Args:
        input_text (str): Входной текст для очистки.

    Returns:
        str: Очищенный текст.
    """
    # HTML-теги: первый шаг - удалить из входного текста все HTML-теги
    clean_text = re.sub('<[^<]+?>', '', input_text)

    # URL и ссылки: далее - удаляем из текста все URL и ссылки
    clean_text = re.sub(r'http\S+', '', clean_text)

    # Эмоджи и эмотиконы: используем собственную функцию для преобразования эмоджи в текст
    clean_text = emojis_words(clean_text)

    # Приводим все входные данные к нижнему регистру
    clean_text = clean_text.lower()

    # Убираем все пробелы
    # Так как все данные теперь представлены словами - удалим пробелы
    clean_text = re.sub('\s+', ' ', clean_text)

    # Убираем специальные символы: избавляемся от всего, что не является "словами"
    clean_text = re.sub('[^а-яА-ЯёЁ0-9\s]', '', clean_text)

    # Записываем числа прописью: 100 превращается в "сто" (для компьютера)
    temp = inflect.engine()
    words = []
    for word in clean_text.split():
        if word.isdigit():
            words.append(num2words(int(word), lang='ru'))
        else:
            words.append(word)
    clean_text = ' '.join(words)

    # Стоп-слова: удаление стоп-слов - это стандартная практика очистки текстов
    stop_words = set(stopwords.words('russian'))
    words = [word for word in words if word not in stop_words]

    # Лемматизация слов
   # morph = pymorphy2.MorphAnalyzer()
   # lemmas = [morph.parse(word)[0].normal_form for word in words]
   # clean_text = ' '.join(lemmas)

    # Знаки препинания: далее - удаляем из текста все знаки препинания
    clean_text = re.sub(r'[^\w\s]', '', clean_text)

    # И наконец - возвращаем очищенный текст
    return clean_text


# Функция для преобразования эмоджи в слова
def emojis_words(text):

    # Модуль emoji: преобразование эмоджи в их словесные описания
    clean_text = emoji.demojize(text, delimiters=(" ", " "))

    # Редактирование текста путём замены ":" и" _", а так же - путём добавления пробела между отдельными словами
    clean_text = clean_text.replace(":", "").replace("_", " ")

    return clean_text

In [24]:
def preproc_text(text):
    res = str(text).strip()
    res = re.sub(r"\s+", " ", res)
    return res

def build_data(data_q, data_ans):
    data = []
    for idx, texts in enumerate(data_q):
        question = clean_text(texts)
        answer = clean_text(data_ans.iloc[idx])
        res = '\nx:' + question + '\ny:' + answer
        data.append(res)
    return data

In [25]:
from tqdm import tqdm

# Форматирование тренировочных данных с прогресс-баром
train_data = []
for idx, texts in tqdm(enumerate(train_q), total=len(train_q), desc='Processing train data'):
    question = clean_text(texts)
    answer = clean_text(train_ans.iloc[idx])
    res = '\nx:' + question + '\ny:' + answer
    train_data.append(res)

# Форматирование тестовых данных с прогресс-баром
test_data = []
for idx, texts in tqdm(enumerate(test_q), total=len(test_q), desc='Processing test data'):
    question = clean_text(texts)
    answer = clean_text(test_ans.iloc[idx])
    res = '\nx:' + question + '\ny:' + answer
    test_data.append(res)


Processing test data: 100%|██████████| 5577/5577 [00:09<00:00, 578.96it/s]


In [ ]:
#train_data = build_data(train_q, train_ans)
#test_data = build_data(test_q, test_ans)

Этот код устанавливает токен, который будет использоваться при заполнении (padding) последовательностей текстовых данных при обучении нейронных сетей. Это делается с помощью установки значения eos_token (токена конца последовательности) как значения pad_token (токена-заполнителя).

Обычно в обучающем наборе данных бывают последовательности, которые имеют разную длину. Для обеспечения удобства и эффективности обучения модели, все последовательности должны быть одинаковой длины. Один из подходов к достижению этой цели - добавление токенов-заполнителей в конец последовательности до тех пор, пока ее длина не станет равной максимальной длине в обучающем наборе данных.

Установка pad_token в значение eos_token означает, что при заполнении последовательности токеном-заполнителем будет использоваться токен, обозначающий конец последовательности. Это может быть полезно, когда необходимо предотвратить модель от генерации ненужной информации после конца последовательности, ведь такие лишние токены могут негативно сказаться на точности модели.

In [26]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
#tokenizer.add_special_tokens({'pad_token': '[PAD]'})

Токенизируем тренировочный и тестовый датасеты:

In [27]:
train_encodings = tokenizer(train_data, padding='max_length', truncation=True, max_length=512, return_tensors='pt')
test_encodings = tokenizer(test_data, padding='max_length', truncation=True, max_length=512, return_tensors='pt')

In [28]:
class QAcookingDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = QAcookingDataset(train_encodings)
test_dataset = QAcookingDataset(test_encodings)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [30]:
# Настройка тренировочных аргументов
training_args = TrainingArguments(
    output_dir="./models", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )

# Задание значения max_steps перед инициализацией тренера
#training_args.max_steps = len(train_dataset) // training_args.per_device_train_batch_size * training_args.num_train_epochs

# Определение тренера
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
    eval_dataset=test_dataset
)




In [31]:
# Обучение модели
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-28-9ec896e8de93>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,2.884800
1000,2.666500
1500,2.606700
2000,2.542800
2500,2.499800
3000,2.476100
3500,2.434100
4000,2.404000
4500,2.377000
5000,2.353400


<ipython-input-28-9ec896e8de93>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-28-9ec896e8de93>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-28-9ec896e8de93>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-28-9ec896e8de93>:6: UserWarning: To copy construct from a 

TrainOutput(global_step=16731, training_loss=2.255385596485262, metrics={'train_runtime': 9840.6583, 'train_samples_per_second': 6.8, 'train_steps_per_second': 1.7, 'total_flos': 1.7485924073472e+16, 'train_loss': 2.255385596485262, 'epoch': 3.0})

Попробовала дообучить rubert-tiny2 на датасете blinoff/medical_qa_ru_data.
Обучать на CPU это боль)) в колабе не смогла дождаться, обучение прервалось через 6 часов. Сократила до 2-х эпох и урезала датасет. Взяла всего 10%, передала на cuda, обучила минут за 10-ть.  Дообученная модель выдает слишком короткие ответы.

Сомневаюсь в правильности подготовки данных (вопрос-ответ) для обучения и вывода предсказаний.

Сохраним модель на диск:

In [32]:
tokenizer.save_pretrained(path_m + 'tokenizer_qa_cooking')
model.save_pretrained(path_m + 'model_qa_cooking')

In [33]:
tokenizer = AutoTokenizer.from_pretrained(path_m + 'tokenizer_qa_cooking')
model = AutoModelForCausalLM.from_pretrained(path_m + 'model_qa_cooking')

Функция для ведения диалога. Модель "держит в голове" контекст последних 10-ти сообщений.

In [34]:
def respond_to_dialog(texts):
    prefix = '\nx:'
    for i, t in enumerate(texts):
        prefix += t
        prefix += '\nx:' if i % 2 == 1 else '\ny:'
    tokens = tokenizer(prefix, return_tensors='pt')
    tokens = {k: v.to(model.device) for k, v in tokens.items()}
    end_token_id = tokenizer.encode('\n')[0]
    size = tokens['input_ids'].shape[1]
    output = model.generate(
        **tokens,
        eos_token_id=end_token_id,
        do_sample=True,
        max_length=size+128,
        repetition_penalty=3.2,
        temperature=1,
        num_beams=3,
        length_penalty=0.01,
        pad_token_id=tokenizer.eos_token_id
    )
    decoded = tokenizer.decode(output[0])
    result = re.findall(r'\ny:(.+)', decoded)[-1]
    return result.strip()

Протестируем генератор:

In [35]:
seed = input('Начните диалог с ботом любой фразой\n')
history = [seed]
while True:
    result = respond_to_dialog(history[-10:])
    next_sentence = input(result + '\n')
    if next_sentence == 'Стоп':
        break
    history.append(result)
    history.append(next_sentence)

Начните диалог с ботом любой фразой
борщ
один свеклу очистить и нарезать соломкой два капусту нашинковать три морковь почистить и натереть на крупной терке четыре лук мелко порубить пять в кастрюлю налить растительное масло положить лавровый лист горошины душистого перца горошины душистого перца горошины душистого перца горошины душистого перца влить уксус посолить поперчить довести до кипения уменьшить огонь накрыть крышкой и тушить десять минут шесть готовый борщ разлить по тарелкам посыпать рубленой зеленью петрушки или укропом семь подавать со сметаной вареным яйцом квашеной капустой или пассерованными ово
как приготовить блины
один разогреть сковороду смазать ее подсолнечным маслом обжарить тонкие блинчики с двух сторон до румяной корочки затем перевернуть их на другую сторону жарить еще двенадцать минуты пока они не станут золотистокоричневыми восемь готовые блинчики полить растопленным сливочным маслом и украсить зеленью петрушки или укропом девять подать со сметаной вареным яйц

Ура, препроцессинг текста помог, модель генерирует не идеальные, но какие-никание рецепты)

Вроде работает ) не так хорошо как хотелось бы, конечно